## 커스텀 레이어
##### url: https://www.tensorflow.org/tutorials/customization/custom_layers

- 목차
    - 레이어: 유용한 연산들의 일반적 집합
    - 커스텀 레이어 구현하기
    - 모델: 레이어로 구성되는 모델
     
신경망을 만들기 위해 고수준 API로써 `tf.keras`를 사용하기를 추천합니다.<br>
이는, 대부분의 Tensorflow API는 '즉시 실행'(eager execution)으로 사용할 수 있기 때문입니다.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

In [3]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


## 레이어: 유용한 연산들의 일반적인 집합
머신러닝 모델을 작성할 때 대부분의 경우, 독립적인 연산과 독립적인 변수를 다루는 것보다 더 높은 수준의 추상화로 작동하길 원합니다.<br><br>
많은 머신러닝 모델은 비교적 간단한 레이어의 적층이나 구성으로서 표현되어집니다. 또한, Tensorflow는 일반적으로 알려진 많은 레이어 뿐만아니라 처음부터 구현하거나 기존의 레이어의 조합으로써 사용자의 어플리케이션에 특정화된 레이어를 제공합니다. <br><br>
Tensorflow는 tf.keras 패키지에 Keras API의 전체를 포함하고 있으며, 모델을 작성할 때 Keras 레이어는 매우 유용합니다.

In [4]:
# tf.keras.layers 패키지에 들어있는 레이어는 객체입니다. 레이어를 생성하기 위해서는,
# 간단히 객체를 생성하면 됩니다. 대부분의 레이어는 첫 번째 인자로써 출력 차원의 수 / 채널을 입력받습니다.
layer = tf.keras.layers.Dense(100)
# 입력 차원의 수는 종종 필요하지 않을 수 있습니다. 레이어가 처음 사용되어질 때
# 추론될 수 있기 때문입니다. 하지만 수동으로 입력해서 해당 값을 제공할 수 있다면
# 모델의 복잡도에 도움이될 수 있습니다.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

존재하는 레이어의 전체 목록은 [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers) 에서 확인할 수 있습니다. Dense(a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout 등과 같은 많은 레이어가 있습니다.

In [5]:
# 레이어를 사용하기 위해서는 간단하게 호출하면 됩니다.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
# 레이어는 많은 유용한 메소드를 가지고 있습니다. 예를 들어, `layer.variable`과 `layer.trainable_variables`를 사용하는 학습 가능한 변수를
# 확인할 수 있습니다, fully-connected layer의 경우 가중치와 편향 값을 가지고 있습니다.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.55515075,  0.54755193, -0.16490513, -0.30627328,  0.23116928,
         -0.06486195,  0.6190699 , -0.03247744, -0.60940635,  0.39874834],
        [-0.34989235,  0.08193505, -0.07927018,  0.23815638,  0.31500757,
         -0.00096458,  0.42698282, -0.32687965,  0.0357824 , -0.37204704],
        [ 0.628364  ,  0.36495405,  0.08304095,  0.3040583 , -0.60080457,
          0.09846711, -0.5871453 , -0.11027646,  0.35822403,  0.27251488],
        [ 0.6187597 ,  0.16909814, -0.60023206, -0.31183225,  0.33960474,
          0.04084474, -0.2698717 , -0.40194717, -0.48740828,  0.34618306],
        [-0.38899875, -0.27183875,  0.2485581 ,  0.20124352, -0.3794446 ,
         -0.48292458, -0.4629293 ,  0.29840344, -0.08070928, -0.6279386 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [7]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.55515075,  0.54755193, -0.16490513, -0.30627328,  0.23116928,
         -0.06486195,  0.6190699 , -0.03247744, -0.60940635,  0.39874834],
        [-0.34989235,  0.08193505, -0.07927018,  0.23815638,  0.31500757,
         -0.00096458,  0.42698282, -0.32687965,  0.0357824 , -0.37204704],
        [ 0.628364  ,  0.36495405,  0.08304095,  0.3040583 , -0.60080457,
          0.09846711, -0.5871453 , -0.11027646,  0.35822403,  0.27251488],
        [ 0.6187597 ,  0.16909814, -0.60023206, -0.31183225,  0.33960474,
          0.04084474, -0.2698717 , -0.40194717, -0.48740828,  0.34618306],
        [-0.38899875, -0.27183875,  0.2485581 ,  0.20124352, -0.3794446 ,
         -0.48292458, -0.4629293 ,  0.29840344, -0.08070928, -0.6279386 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)